# ベイズ推論による機械学習入門
## 3.5 線型回帰

### Example

In [ ]:
using LinearAlgebra
using Distributions
using Plots
gr()

In [ ]:
M = 4 # dimensions
N = 100
xmin = -1
xmax = 1
x = range(xmin, xmax, length=N)
x_sample = xmin .+ (xmax-xmin) .* rand(10)

lambda = 10
epsilon = Normal(0, lambda^(-1))

# weight
m = zeros(M)
l_lambda = Matrix{Float64}(I, 4, 4)

In [ ]:
# generate noise
ep = rand(epsilon, N)
plot(ep, seriestype=:scatter)

In [ ]:
# smapling weight
w = []
l_lambda_inv = inv(l_lambda)
for j in 1:M
    dist_w = Normal(m[j], l_lambda_inv[j, j])
    push!(w, rand(dist_w, 1)[1])
end
println("w:", w)

In [ ]:
# plot data
y_true = []
y_line = []

# true distribution
for i in 1:N
    input = [1, x[i], x[i]^2, x[i]^3]
    y_buff = dot(input, w)
    push!(y_line, y_buff)
end

# measured data
for i in 1:10
    input = [1, x_sample[i], x_sample[i]^2, x_sample[i]^3]
    dist_y = Normal(dot(input, w), lambda^(-1))
    push!(y_true, rand(dist_y, 1)[1]+ep[i])
end

plot(x, y_line, label="true")
plot!(x_sample, y_true, seriestype=:scatter, label="measured")

----
### 事後分布

$$ p(\vec{w}|\vec{Y},\vec{X})=N(\vec{w}|\vec{\hat{m}},\vec{\hat{\Lambda}}^{-1}) $$
$$ \vec{\hat{\Lambda}} = \lambda \sum^{N}_{n=1} \vec{x}_{n}\vec{x}^{T}_{n} + \vec{\Lambda} $$
$$ \vec{\hat{m}} = \vec{\hat{\Lambda}}^{-1} \lambda (\sum^{N}_{n=1} y_{n}\vec{x}_{n} + \vec{\Lambda}\vec{m}) $$

In [ ]:
println("lambda:", lambda)
l_lambda

In [ ]:
# calculation large lambda hat
input = [1, x_sample[1:end], x_sample[1:end].^2, x_sample[1:end].^3]
sum_x = sum(dot(input, input))
println("sum_x:", sum_x)

l_lambda_hat = lambda * sum_x .+ l_lambda
l_lambda_hat

In [ ]:
inv(l_lambda_hat)

In [ ]:
# calculation m_hat
sum_xy = zeros(M)
for (i, xm) in enumerate(x_sample)
    input = [1, xm, xm^2, xm^3]
    sum_xy = y_true[i] * input .+ sum_xy
end
println("sum_xy:", sum_xy)

m_hat = inv(l_lambda_hat) * lambda * sum_xy + l_lambda * m
print("m_hat:", m_hat)

In [ ]:
lambda * sum_xy

In [ ]:
# calc weight
println("w_true:", w)
w_pred = []
l_lambda_hat_inv = inv(l_lambda_hat)
for j in 1:M
    dist_w = Normal(m_hat[j], l_lambda_hat_inv[j, j])
    push!(w_pred, rand(dist_w, 1)[1])
end
println("w_pred:", w_pred)

In [ ]:
y_pred_line = []
for i in 1:N
    input = [1, x[i], x[i]^2, x[i]^3]
    y_buff = dot(input, w_pred)
    push!(y_pred_line, y_buff)
end

plot!(x, y_pred_line, seriescolor=:green, label="predict")

----
### 予測分布